# Homework 7

In [1]:
# imports
import numpy as np
from importlib import reload

from oceanpy import constants
import unyt
from oceanpy.forces import coriolis as ofc

from bokeh import plotting
from bokeh import models

from IPython.display import Image
from IPython.core.display import HTML 

In [2]:
def set_fontsize(p, fsz):
    '''
    Parameters
    ----------
    ax : Bokeh plot class
    fsz : float
      Font size
    '''
    p.xaxis.axis_label_text_font_size = '{:d}pt'.format(fsz)
    p.xaxis.major_label_text_font_size = "{:d}pt".format(fsz)
    #
    p.yaxis.axis_label_text_font_size = '{:d}pt'.format(fsz)
    p.yaxis.major_label_text_font_size = "{:d}pt".format(fsz)

# 1. Horizontal Current

## Expression:  $du/dz = c$, with c a constant

## a) Integrate the expression in $z$ between a depth of 1000m and a shallower depth $d$.

### $\Delta u = \int\limits_{-1000 \, \rm m}^{z'} (du/dz) dz = c \, (z' + 1000\,\rm m)$ 

## b) Suppose $c = 10^{-3} \, \rm s^{-1}$ and $u=0$ at $z=1000$m.  What is $u$ at 900m?

### We can use our equation to calculate $\Delta u = 100\,{\rm m} \times 10^{-3} \, {\rm s^{-1}} = 0.1 \, {\rm m/s}$

### Signs are confusing here, but if $z$ decreases with depth, then this means $u = 0.1 \, \rm m/s$ at 900 m depth.

### For $z=-500\, \rm m$, $\Delta u = 0.5 \, \rm m /s$.  Therefore $u = 0.5 \, \rm m/s$ at 500 m.

### Likewise, $\Delta u = 1\, \rm m/s$ for $z=0$m.  And $u = 1 \, \rm m/s$ at the surface.

## c) Recalculate for a zonal velocity at 1000m of 0.5 m/s.

### All of the $\Delta u$ values remain.  But the currents are offset by 0.5 m/s.

### i.e. $u = 0.6 \, \rm m/s$ at 900m, $u = 1 \, \rm m/s$ at 500m and $u = 1.5 \, \rm m/s$ at the surface.

## d) Let $c= 5 \times 10^{-4} \, \rm s^{-1}$ for $-1000 {\rm m} < z < -500 {\rm m}$ and $10^{-3} \, \rm s^{-1}$ for $z > -500$\, m.  What is the surface velocity relative to the 1000m depth?

### Our expression for $\Delta u$ now has two components.  Here it is assuming $z > -500 \, \rm m$:

### $\Delta u = 5 \times 10^{-4} (500 \rm m/s) + 10^{-3} \, {\rm s^{-1}} (z' + 500 \, \rm m)$

In [4]:
5e-4 * 500 + 1e-3 * 500

0.75

### For the surface, we recover $\Delta u = 0.75 \, \rm m/s$, i.e. relative to 1000 m depth.

----

# 2. Thermal Wind equations

## a) What are the thermal wind equations and describe in words

### Here are the equations:

### $-f \partial v / \partial z = (g / \rho_0) \, \partial \rho / \partial x$

###  $f \partial u / \partial z = (g / \rho_0) \, \partial \rho / \partial y$

### These equations describe the change in geostrophic currents with depth due to horizontal density variations.    Such variations often arise from temperature variations across the current.

## b) What is the level of no motion, and why is it a shortcoming?

### The thermal wind equations allow one to measure the change in current velocity with depth, but do not yield the absolute  motion.  Standard practice is to assume the velocities are zero at a reference depth, the "level of no motion".  This is frequently asserted without measuring, and in some cases is significantly inaccurate.

----

# 3. Potential Temperature for a zonal section

## Assume a linear equation of state:

## $\rho = \rho_0 - \alpha (T - T_0)$

## with $\rho_0 = 1027 \, \rm kg/m^3$, $T_0 = 10^\circ \, \rm C$, and $\alpha = 0.15 \, \rm kg/m^3/K$.

### Assume a level of no motion at depth of 3000m.

## a) What is the direction of surface flow above about $100^\circ$ W?

### In the upper 1000m, at $100^\circ$W there is a negative temperature gradient towards the E ($\partial T / \partial x < 0$).  This means $\partial \rho / \partial x > 0$.

### From our Thermal wind equations, this means $\partial v / \partial z = -(g/\rho_0) |\partial \rho / \partial x| / f$

### Given we are in the Southern hemisphere, $f < 0$ and we have $\partial v / \partial z > 0$.

### So for $v = 0$ at $z = -3000$m, we have $v > 0$ at the surface.  This means a current towards the North!

## b) What is the direction of deep flow at 5000m depth at about $175^\circ$W?

### Below 3000m at $175^\circ$W, $\partial T/\partial x > 0$ and therefore $\partial v/\partial z < 0$.

### If $v = 0$ at $z=-3000$m, then $v>0$ at $z=-5000$m.  Again, a current towards the North.

## c) Make a rough estimate of the speed for this deep flow.

### We need to estimate $\partial T / \partial x$ and apply the Equation of state to estimate $\partial \rho/\partial x$.

### Eyeballing the figure, I estimate $\Delta T \approx 0.2^\circ$K over $30^\circ$ longitude (3000 km).

### This gives $\Delta T / \Delta x \approx 7 \times 10^{-8}$ K/m.

### And, $\Delta \rho / \Delta x \approx 10^{-8}$ kg/m$^4$

### At $\phi = 35^\circ$S, $f=-8 \times 10^{-5} \, \rm s^{-1}$.

In [15]:
DT = 0.2 * unyt.K
Dx = 3000 * unyt.km
alpha = 0.15 * unyt.kg / unyt.K / unyt.m**3
rho_0 = 1027 * unyt.kg / unyt.m**3

In [14]:
Drho_Dx = -alpha * DT/Dx
Drho_Dx.to('kg/m**4')

unyt_quantity(-1.e-08, 'kg/m**4')

In [16]:
f = ofc.coriolis_parameter(-35*unyt.deg)
f.to('s**-1')

unyt_quantity(-8.36518044e-05, '1/s')

In [17]:
g = 9.8 * unyt.m / unyt.s**2
Dv_Dz = -1 * Drho_Dx * (g/rho_0) / f
Dv_Dz

unyt_quantity(-1.14072332e-06, '1/s')

### We estimate $\partial v / \partial z \approx -10^{-6} \, \rm s^{-1}$.

### For $\Delta z = 2000$m, we have $\Delta v = 0.002$ m/s.

In [19]:
Dv = Dv_Dz * 2000*unyt.m
Dv

unyt_quantity(-0.00228145, 'm/s')